In [1]:
import threading

In [2]:
threading.active_count()

8

In [4]:
threading.current_thread()

<_MainThread(MainThread, started 4436334080)>

In [5]:
threading.enumerate()

[<_MainThread(MainThread, started 4436334080)>,
 <Thread(IOPub, started daemon 123145560215552)>,
 <Heartbeat(Heartbeat, started daemon 123145577005056)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145594867712)>,
 <Thread(Thread-4 (_watch_pipe_fd), started daemon 123145611657216)>,
 <ControlThread(Control, started daemon 123145628446720)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145645236224)>,
 <ParentPollerUnix(Thread-2, started daemon 123145662562304)>]

In [6]:
main_th = threading.current_thread()

In [10]:
main_th.is_alive(), main_th.daemon, main_th.name, main_th.ident, main_th.native_id

(True, False, 'MainThread', 4436334080, 14190344)

In [18]:
def print_thread_data(*args, **kwargs):
    import time
    time.sleep(4)

    th = threading.current_thread()
    print(f"{th.name=}, {th.is_alive()=}, {th.native_id=}, {args=}, {kwargs=}")
    import time
    time.sleep(4)


th = threading.Thread(
    name="th_print",
    target=print_thread_data,
    args=(42,),
    kwargs={"a": 11, "b": 22},
)
th.start()
th.join()

print("END")

END


In [35]:
LIMIT = 100_000
AMOUNT = 1000

def countdown(n):
    global jobs_finished
    jobs_finished += 1

    while n > 0:
        n -= 1
        

def do_work(amount, n=LIMIT):
    for i in range(amount):
        countdown(n)

In [36]:
%%time

jobs_finished = 0

do_work(AMOUNT)

print(f"{jobs_finished=}")

jobs_finished=1000
CPU times: user 4.62 s, sys: 61.7 ms, total: 4.68 s
Wall time: 5.19 s


In [37]:
%%time

jobs_finished = 0

th = threading.Thread(
    target=do_work,
    args=(AMOUNT,),
)
th.start()
th.join()

print(f"{jobs_finished=}")

jobs_finished=1000
CPU times: user 5.35 s, sys: 97.3 ms, total: 5.44 s
Wall time: 6.9 s


In [39]:
%%time

N_THREADS = 1000

jobs_finished = 0

threads = [
    threading.Thread(
        target=do_work,
        args=(AMOUNT // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
print(f"{jobs_finished=}")

jobs_finished=1000
CPU times: user 4.46 s, sys: 107 ms, total: 4.57 s
Wall time: 4.93 s


In [50]:
from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
AMOUNT = 30


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(amount, url=URL):
    for _ in range(amount):
        resp = fetch_url(url)

In [51]:
%%time

fetch_batch_urls(AMOUNT)

CPU times: user 291 ms, sys: 33.4 ms, total: 325 ms
Wall time: 4.86 s


In [52]:
%%time


th = threading.Thread(
    target=fetch_batch_urls,
    args=(AMOUNT,),
)
th.start()
th.join()

CPU times: user 298 ms, sys: 34.7 ms, total: 333 ms
Wall time: 5.79 s


In [57]:
%%time

N_THREADS = 5

threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(AMOUNT // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 299 ms, sys: 36.8 ms, total: 336 ms
Wall time: 1.1 s


In [70]:
%%time

import time


AMOUNT = 100000
N_THREADS = 10


def danger_update():
    global jobs_finished

    temp = jobs_finished  # 3
    temp += 1

    time.sleep(0.00000001)

    jobs_finished = temp  # 5


def update_batch(n):
    for _ in range(n):
        danger_update()


jobs_finished = 0


threads = [
    threading.Thread(
        target=update_batch,
        args=(AMOUNT // N_THREADS,),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

    
expected = AMOUNT

print(f"{jobs_finished=}, {expected=}")

jobs_finished=10006, expected=100000
CPU times: user 328 ms, sys: 866 ms, total: 1.19 s
Wall time: 841 ms


In [73]:
%%time

import time


AMOUNT = 100000
N_THREADS = 10


lock = threading.Lock()


def danger_update(lock):
    global jobs_finished
    
    lock.acquire()

    temp = jobs_finished
    temp += 1

    time.sleep(0.00000001)
    jobs_finished = temp
    
    lock.release()


def update_batch(n, lock):
    for _ in range(n):
        danger_update(lock)


jobs_finished = 0


threads = [
    threading.Thread(
        target=update_batch,
        args=(AMOUNT // N_THREADS, lock),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()


expected = AMOUNT

print(f"{jobs_finished=}, {expected=}")

jobs_finished=100000, expected=100000
CPU times: user 328 ms, sys: 499 ms, total: 827 ms
Wall time: 1.04 s


In [77]:
%%time

from urllib.request import urlopen

URL = "https://ru.wikipedia.org/wiki/Python"
AMOUNT = 30
N_THREADS = 5

sem = threading.Semaphore(3)


def fetch_url(url, sem):
    with sem:
        resp = urlopen(url)
    return resp


def fetch_batch_urls(amount, sem, url=URL):
    for _ in range(amount):
        resp = fetch_url(url, sem)
        
        
threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(AMOUNT // N_THREADS, sem),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 292 ms, sys: 34.3 ms, total: 326 ms
Wall time: 1.76 s


In [89]:
%%time

from urllib.request import urlopen
import queue

URL = "https://ru.wikipedia.org/wiki/Python"
AMOUNT = 30
N_THREADS = 5

sem = threading.Semaphore(3)
que = queue.Queue(maxsize=10)


def fetch_url(sem, que):
    while True:
        try:
            # url = que.get(timeout=1)
            url = que.get()
        except queue.Empty:
            continue

        if url is None:
            print("THREAD ENDS")
            que.put(None)
            break

        with sem:
            resp = urlopen(url)
        
        
threads = [
    threading.Thread(
        target=fetch_url,
        args=(sem, que),
    )
    for _ in range(N_THREADS)
]

for th in threads:
    th.start()


for _ in range(AMOUNT):
    que.put(URL)  # 10

que.put(None)


for th in threads:
    th.join()
    
    
print(que.qsize())

THREAD ENDS
THREAD ENDS
THREAD ENDS
THREAD ENDS
THREAD ENDS
1
CPU times: user 287 ms, sys: 34 ms, total: 321 ms
Wall time: 1.72 s


In [92]:
import multiprocessing


class A:
    def __init__(self, a):
        self.a = a


def update_data(data):
    pass
#     data["qwerty"] = 2 ** 100
#     data["a"] = A(10)


data = {}

proc = multiprocessing.Process(target=update_data, args=(data,))

proc.start()
proc.join()

print(f"{data=}")

data={}


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'update_data' on <module '__main__' (built-in)>


In [93]:
import socket

In [ ]:
# server.py
sock = socket.socket()
sock.bind(("localhost", 5000))
sock.listen(5)
while True:
    addr, client_sock = sock.accept()


# client.py
sock = socket.socket()
sock.connect(("localhost", 5000))
sock.send(URL)
data = sock.recv()